## ASTR 21000, Winter 2020

## Homework Assignment 5 (55 points + 20 extra-credit exercise points)

### Distributed: Monday, March 2

### Due: Friday,  March 6 at 11:59pm 

import packages needed by the codes below. Run this cell first before using these codes. 

In [41]:
import numpy as np

# use jupyter "magic" command to tell it to embed plot into the notebook 
import matplotlib.pyplot as plt
%matplotlib inline

from codes.plot_utils import plot_pretty
plot_pretty(fontsize=12)

### Introduction 

In this homework you will redo the analysis you did in homework 3 using supernovae to constrain model parameters but with a different approach. There are two differences in the approach you will use in this exercise: 

1) you will work with the *likelihood function* instead of $\chi^2$. The likelihood function and the rationale for its use, as well as relation of $\chi^2$ to this function were introduced in the lab and you can find the description in <a href="07_model_parameter_inference.ipynb"><tt>07_model_parameter_inference.ipynb</tt></a> notebook. More precisely, you will work with the likelihood function and prior probability distributions for model parameters, which combined give the pdf called *posterior distribution function*. 

2) you will use the model that abandons assumption that supernovae type Ia are "standard candles" (i.e. have the same absolute magnitude value $M_0$). Instead, the absolute magnitude of each supernova will be corrected using two of its properties measured in observations. This requires two additional parameters, which brings the number of model parameters from 3 to 5. 

Thus, we have to map out the likelihood in 5-dimensional model parameter space, which cannot be easily done by sampling on a grid as could be done for 3D parameter space in homework 3. Thus, in the two exercises in this homework you will first find the "best fit" model parameters in the first exercise and then will learn how to estimate uncertainties by sampling 5-dimensional posterior distribution. 

### Exercise 1: using supernova type Ia sample to measure the average density of matter and vacuum energy in our Universe: final analysis using likelihood function and priors (40 points)

#### What are you learning in this exercise

1. How to esimate parameters using likelihood, priors, and minimization of the -posterior distribution

2. How to map multi-dimensional posterior distribution using MCMC sampling and how to estimate confidence levels of parameters. 

#### Background:

The model described in <a href="hw03.ipynb"><tt>hw03.ipynb</tt></a> has 3 parameters: $\Omega_{\rm m0}$, $\Omega_\Lambda$, $\tilde{M_0}$ and predicts the difference between observed and model distance modulus to be: 

\begin{eqnarray}
\Delta\mu=\mu_{\rm obs} - \mu(z,\Omega_{\rm m0},\Omega_\Lambda) &=&m_{\rm SNIa} - M_0 - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - 5\log_{10}c + 5\log_{10}H_0 - 25\\
&=& m_{\rm SNIa} - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - \tilde{M}_0, 
\end{eqnarray}

where 

$$ \tilde{M}_0 = \left(M_0 + 5\log_{10}\,\frac{c}{H_0}+25\right).$$

This model assumes that the same absolute magnitude, $M_0$ (a measure of intrinsic supernova luminosity), is the same for all supernova. Objects for which this is true are called "standard candles" in astronomy. This is not a bad assumption. For decades astronomers did think that supernovae type Ia are standard candles. However, about 30 years ago it was shown that this is not quite true. Intrinsic absolute magnitude of supernovae of this type varies, but the variation occurs not randomly, but systematically as a function of supernova color (difference of magnitudes in different bands) and the characteristic time it takes for a supernova brightness to fade from its peak brightness. 

Both of these properties are observable and are characterized by the color parameter, $c$, related to supernova color at peak brightness and lightcurve stretch parameter, $x_1$. Astronomers found that the parameter nearly constant for all supernovae is not $M_0$ ($M_0$ varies), but the combination $M_0 - \alpha x_1 + \beta c$, where $\alpha$ and $\beta$ are parameters that are constant for all supernovae. $M_0$,  and $\alpha$ and $\beta$ are *nuisance* parameters of the model and have to be estimated from the data itself. 

Thus, we can modify our model for the distance modulus by including these two extra parameters: 

$$
\Delta\mu=\mu_{\rm obs} - \mu(z,\Omega_{\rm m0},\Omega_\Lambda)= m_{\rm SNIa} - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - \tilde{M}_0 + \alpha x_1 - \beta c,$$ 

with the vector of 5 parameters $\mathbf{x}=\{\Omega_{\rm m0},\Omega_\Lambda, \tilde{M}_0, \alpha,\beta\}$ and $x_1$ and $c$ stretch and color parameters measured for each supernova. 

These parameters are available in the data file that we've been using along with their observational uncertainties and can be read as shown below along with other quantities we've used previously. 

### Observational sample of supernovae type Ia

We will use the same sample of 740 supernovae published in a study of 
<a href="https://ui.adsabs.harvard.edu/abs/2014A%26A...568A..22B/abstract">Betoule et al. (2014)</a>. The file containing the sample is available in the course repository and was downloaded from <a href="http://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html">here</a>. However, we will now also use $x_1$ and $c$ stretch and color parameters measured for each supernova and their uncertainties, as shown below. 

In [58]:
#zCMB, mB, emB were used before in hw 3
# x1 and ex1 are stretch parameter measured for each SN and its uncertainty
# csn and ecsn are color parameter and its uncertainty
zCMB, mB, emB, x1, ex1, csn, ecsn = np.loadtxt('data/jla_lcparams.txt', 
                                               usecols=(1, 4, 5, 6, 7, 8, 9), unpack=True)

print("read sample of %d supernovae..."%(np.size(zCMB)))

read sample of 740 supernovae...


#### Group work

If you feel that your group work in hw 3 was productive, I encourage you to continue working with your group. You can also form a new group, if you wish. In this type of exercise it is helpful to discuss approach and results in a group and such collaborative work is encouraged. 

**Task 1a. (10 points)** Write a routine that computes $\ln L$ for this new 5-parameter model, where 

$$\ln L(\mathbf{y}\vert\mathbf{x}) = -\frac{1}{2}\,\sum\limits_{i=0}^{N_{\rm SN}-1}\frac{\Delta\mu^2}{\sigma_{\Delta\mu,i}^2}$$

and $\sigma_{\Delta\mu,i}^2$ is total uncertainty of the observational estimate of the distance modulus that accounts for uncertainties in $m_B$, $x_1$, and $c$, which by rules of error propagation is: 

$$\sigma_{\Delta\mu,i}^2 = \sigma_{m_B}^2 + \alpha^2\sigma_{x1}^2 + \beta^2\sigma_c^2,$$

which means that the error also depends on the two model parameters, $\alpha$ and $\beta$. 

**Task 1b. (8 points)** Test your routine by using the differential evolution routine you implemented in hw 4, or <a href="https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.differential_evolution.html"><tt>scipy.minimize.differential_evolution</tt></a> routine to find parameters that minimize $-2\ln L$ and print them out. 

To define bounds, you can use ranges for $\Omega_{\rm m0}$ and $\Omega_\Lambda$ of $[0,1]$ and for $\tilde{M}_0$ say $[20, 28]$. For $\alpha$ good choice for range is $[0.05, 0.3]$ and for $\beta$: $[1., 5.]$. Feel free to experiment with these ranges. 

The values for $\Omega_{\rm m0}$ and $\tilde{M}_0$ you should get should be quite similar (although not exactly the same) as in hw 3, but $\Omega_\Lambda$ should change quite a bit. 
Describe the changes from the values derived in hw 3 and compare the parameters you derive.

**Task 1c (2 points).** Compute the reduced $\chi^2$ for the values of the parameters that minimize the $-2\ln L$. What can you conclude about how well the model describes the supernova measurements? How does the value compares to the value obtained in hw3? 

**Task 1d. (10 points)** Write a function that defines natural logarithm of prior pdf for all parameters $\ln p$. Write another function that defines unnormalized posterior pdf, i.e.  combines $\ln L$ and $\ln p$ into $\ln L(\mathbf{y}\vert\mathbf{x}) + \ln p(\mathbf{x})=\ln L(\mathbf{y}\vert\mathbf{x})p(\mathbf{x})$. For $p$ for all parameters you can assume uniform pdf defined between minimum and maximum value you choose for each parameter (these should be $0$ and $1$ for $\Omega_{\rm m0}$ and $\Omega_\Lambda$ not to extrapolate your approximation for $\tilde{d}_L$. The total prior pdf for all parameters is a product of prior pdfs for individual parameters. (4 points)

Use example routine <tt>vectorized_mcmc</tt> from <a href="06d_multid_sampling_mcmc.ipynb"><tt>06d_multid_sampling_mcmc.ipynb</tt></a> notebook or <tt>emcee</tt> package to sample $\ln L(\mathbf{y}\vert\mathbf{x})p(\mathbf{x})$ using MCMC. Plot resulting distributions of chain values for $\Omega_{\rm m0}$ and $\Omega_\Lambda$ along with the confidence contours that correspond to $1-$ and $2-\sigma$ of the Gaussian distribution and enclose $0.6827$ and $0.9545$ of the total posterior probability using <tt>plot_2d_distribution</tt> routine from <tt>codes/plot_utils.py</tt> (6 points). 

Examples of usage of 
<tt>vectorized_mcmc</tt> and <tt>emcee</tt> and <tt>plot_2d_distribution</tt> routine to plot distribution of MCMC samples and confidence contours can be found in <a href="06d_multid_sampling_mcmc.ipynb"><tt>06d_multid_sampling_mcmc.ipynb</tt></a> notebook. 

**Task 1e (10 points).** Experiment with different number of MCMC steps and compute the correlation time for the chain in each case (using <tt>compute_tcorr</tt> routine below). What chain length is acceptable to get a reasonable plot for the posterior distribution and sufficient statistics of nearly independent samples $N_{\rm chain}/t_{\rm corr}$? 

Present the plot of distribution of the MCMC chain samples in the $\Omega_{\rm m0}-\Omega_\Lambda$ along with the contours enclosing $0.6827$ and $0.9545$ of the total posterior probability. Compare them to the results you obtained in hw 3. What are the main differences?

In [ ]:
def auto_corr_func(timeseries, lagmax):
    """
    compute auto correlation function
    """
    ts = np.asarray(timeseries)
    N = np.size(ts) - 1
    ts -= np.average(ts) # Set to mean 0
    corr_func = np.zeros(lagmax)
    for dt in range(lagmax):
        # sum of ts[t+dt]*ts[t]
        corr_func[dt] = (np.dot(timeseries[0:N-dt],timeseries[dt:N])) 
    if (corr_func[0]>0):
        corr_func /= corr_func[0] # normalize
    return corr_func

def compute_tcorr(timeseries,maxcorr):
    """
    compute auto-correlation time
    Parameters:
    -----------
    
    timeseries: 1d vector of values
    maxcorr: maximum autocorrelation lag to consider
    
    Returns:
    tau, mean, sigma: float scalars
        autocorrelation time, mean of the sequence and its rms 
    """
    timeseries = np.copy(timeseries)
    mean = np.average(timeseries)
    corrfxn = auto_corr_func(timeseries,maxcorr)
    tau = np.sum(corrfxn)-1
    var = np.var(timeseries)
    sigma = np.sqrt(var * tau / len(timeseries))
    return tau, mean, sigma


### Exercise 2. Volume of the $D$-dimensional unit sphere using Monte-Carlo method (15 points)

Volume of $D$-dimensional sphere of radius $r$: 

$$V_{\rm D}(r)=\frac{2r^D\pi^{D/2}}{D\,\Gamma(D/2)},$$

where $\Gamma(x)$ is the incomplete gamma function. 

If you are ever in the situation, where you don't recall this formula (I usually don't) and cannot readily access relevant information, you can try to estimate this volume using Monte Carlo method. How accurate would this be? 

#### What are you learning in this exercise?

1. How to use Monte Carlo methods to estimate quantities that may be tricky to estimate with any other methods, especially in many dimensions. 

2. How fractional accuracy of the Monte Carlo estimates scales with the number of samples.


#### Task 2a. Routine to estimate volume of $D$-dimensional sphere using Monte Carlo method (8 points)

Write a routine that estimates $V_{\rm D}(r)$ using Monte Carlo method. As input routine should have <tt>r</tt> - the radius of the sphere, <tt>d</tt> - the number of dimensions, and <tt>nrnd</tt> - the number of random samples to use: 

    def dsphere_volume(r = 1., d=1, nrnd=1):
    
        Parameters:
        -----------
        r: real
            radius of the sphere
        d: integer
            number of dimensions
        nrnd: integer
            number of Monte Carlo samples to use in the estimate
            
        Returns:
        --------
        real number - estimate of the d-dimensional sphere volume


***Hint:*** to understand how to estimate the volume of a sphere, it is useful to think about a ratio of volumes of the sphere and the cube, in which the sphere is inscribed. 

#### Task 2b. Dependence of the estimate accuracy on $D$ (4 points)

Fix <tt>nrnd</tt> and <tt>r</tt> (say to $r=1$), and use your routine to estimate $V_{\rm D}(r)$ for a series of several values of $D$. Plot estimated $V_{\rm D}(r)$ as a function of $D$ as scatter plot and correct $V_{\rm D}(r)$ using the above formula as a line. How does accuracy of the estimate depend on $D$? (plot fractional difference of the estimates and correct values if needed). Provide a brief interpretation/discussion of this result.

Routine computing exact volume of $D$-dimensional sphere for $r=1$ is provided below. 

In [59]:
from scipy.special import gamma

def dsphere_volume_exact(r=1., dim=1):
       return 2. * r**d * np.pi**(0.5*dim) / (gamma(0.5*dim) * dim)


#### Task 2c.  Dependence of the estimate accuracy on <tt>nrnd</tt> (3 points)

Fix $D$ and $r$ to the values you like and compute the absolute value of the fractional difference of your estimate from the correct result, $\epsilon=\vert 1-V_{\rm D,est}/V_{\rm D, exact}\vert$ for a series of <tt>nrnd</tt>. Plot $\epsilon$ as a function of <tt>nrnd</tt>. Discuss the trend you observe and your interpretation of it. 

### Exercise 3 (extra-credit): Implementing the affine-invariant MCMC algorithm of Goodman & Weare (2010) (20 points). 


#### Task.
Implement the "affine-invariant" MCMC algorithm proposed by <a href="http://msp.org/camcos/2010/5-1/p04.xhtml">Goodman & Weare (2010, hereafter GW10)</a> and discussed in <tt>06d_multid_sampling_mcmc.ipynb</tt></a> notebook in the form of a python routine similar to routine <tt>vectorized_mcmc</tt> that you can find in the above notebook (feel free to use <tt>vectorized_mcmc</tt> as a basis for your routine). The GW10 algorithm has quite a few similarities to the differential evolution algorithm. Although implementing it may be a bit more involved, the effort should be comparable to implementing DE routine. 

Your routine should take on input a function calculating the target posterior pdf and a specified number of walkers (chains) initialized in some way (write a separate initialization routine, where initialization could be easily changed).

Specific implementation you should use is given by the Algorithm 2 or Algorithm 3 of <a href="http://adsabs.harvard.edu/abs/2013PASP..125..306F">Foreman-Mackey et al. (2010)</a>. If you implement Algorithm 3, it can be fairly easily parallelized later on using MPI library for parallel execution. Not strictly needed, but you can also think about a way to specify which dimensions of the parameter space can be "walked on" and which are kept fixed at their initial values. This will make the use of the routine convenient for inference when you want to fix certain parameters at the initial values and only fit the rest. (15 points)

***Hint:*** Note that the algorithm requires sampling of $z_r$ from a $g(x)$ distribution that you should have implemented in hw 4.


#### Required test:
Test your implementation of GW10 sampler against the Metropolis algorithm implemented in the <tt>vectorized_mcmc</tt> routine or <tt>emcee</tt> package for a 2D Gaussian pdf. Compare their acceptance ratios. After this test is passed you can also test your routine by sampling unnormalized posterior distribution in exercise 1 above.   (5 points)
